## Tensorflow HuBMAP - Hacking the Kidney competition starter kit:

* https://www.kaggle.com/wrrosa/hubmap-tf-with-tpu-efficientunet-512x512-tfrecs (how to create training and inference tfrecords)
* this notebook (training pipeline)
* https://www.kaggle.com/wrrosa/hubmap-tf-with-tpu-efficientunet-512x512-subm (inference with submission)


# Versions
* V1-V6 init
* V7: 4-CV efficientunetb0 512x512 (LB .834)
* V8: loss bce, fixed dice_coe function for tpu (LB .835)
* V9: efficientunetb1, added oof metrics.json (CV .871, LB .830)
* V10: efficientunetb4 (CV .874, LB .839) 
* V11: 
    * updated files paths (moved files to train folder)
    * efficientunetb7 (memory issue on 3-rd fold)
* V12: P['BATCH_COE'] = 4, efficientunetb7 (CV .858, LB .835)
* V13: efficientunetb4, P['BATCH_COE'] = 8, P['SEED'] = 1 (just rerun nb V10 with best lb and consume tpu quota ;) ) (CV .877, LB .836)
* V14: efficientunetb4, add overlapped tiles and P['EPOCHS'] = 30 and P['SEED'] = 0 (CV .8798, LB .843 (THRESHOLD = 0.5, MIN_OVERLAP = 32); LB .846 (THRESHOLD = .4, MIN_OVERLAP = 32); LB .848 (THRESHOLD = .4, MIN_OVERLAP = 300))
* V15: efficientunetb4, fixed issue with image counting in training filenames, paths to train2 updated, added P['STEPS_COE'], P['TILING'], P['DIM_FROM'] (...)
* V16: efficientunetb4, competition data update, P['STEPS_COE'] = 1, P['NFOLDS'] = 5



# Refferences:
* @marcosnovaes  https://www.kaggle.com/marcosnovaes/hubmap-looking-at-tfrecords and https://www.kaggle.com/marcosnovaes/hubmap-unet-keras-model-fit-with-tpu
* @mgornergoogle https://www.kaggle.com/mgornergoogle/getting-started-with-100-flowers-on-tpu
* @qubvel https://github.com/qubvel/segmentation_models  !! 25 available backbones for each of 4 architectures
* @kool777, @joshi98kishan https://www.kaggle.com/kool777/training-hubmap-eda-tf-keras-tpu
* @cdeotte https://www.kaggle.com/cdeotte/triple-stratified-kfold-with-tfrecords


# Init - parameters, packages, gcs_paths, tpu

In [ ]:
P = {}
P['EPOCHS'] = 30
P['BACKBONE'] = 'efficientnetb4' 
P['NFOLDS'] = 5
P['SEED'] = 0
P['VERBOSE'] = 1
P['DISPLAY_PLOT'] = True 
P['BATCH_COE'] = 8 # BATCH_SIZE = P['BATCH_COE'] * strategy.num_replicas_in_sync

P['TILING'] = [1024,512] # 1024,512 1024,256 1024,128 1536,512 768,384
P['DIM'] = P['TILING'][1]
P['MASK_DIM'] = P['TILING'][0]
P['DIM_FROM'] = P['TILING'][0]

P['LR'] = 5e-4 
P['OVERLAPP'] = True
P['STEPS_COE'] = 1.

import yaml
with open(r'params.yaml', 'w') as file:
    yaml.dump(P, file)

In [ ]:
! pip install segmentation_models -q
%matplotlib inline

import os
os.environ['SM_FRAMEWORK'] = 'tf.keras'
import glob
import segmentation_models as sm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.utils import get_custom_objects

from kaggle_datasets import KaggleDatasets
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
try: # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError: # no TPU found, detect GPUs
    #strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

BATCH_SIZE = P['BATCH_COE'] * strategy.num_replicas_in_sync

print("Number of accelerators: ", strategy.num_replicas_in_sync)
print("BATCH_SIZE: ", str(BATCH_SIZE))

In [ ]:
def get_training_filenames(base_path,nfiles=2):
    training_filenames = []
    for i in range(1,nfiles+1):
        GCS_PATH = KaggleDatasets().get_gcs_path(f'{base_path}-{i}')
        training_filenames += tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
    return training_filenames

In [ ]:
ALL_TRAINING_FILENAMES = get_training_filenames('tfrecords-mask-1024')
if P['OVERLAPP']:
    ALL_TRAINING_FILENAMES2 = get_training_filenames('tfrecords-mask2-1024')

In [ ]:
print(len(ALL_TRAINING_FILENAMES2))
ALL_TRAINING_FILENAMES2

## GCS_PATHS

In [ ]:
# GCS_PATH = KaggleDatasets().get_gcs_path(f'hubmap-tfrecords-1024-{P["DIM"]}')
# ALL_TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
# ALL_TRAINING_FILENAMES

In [ ]:
import re
def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)
print('NUM_TRAINING_IMAGES:' )
if P['OVERLAPP']:
    print(count_data_items(ALL_TRAINING_FILENAMES2)+count_data_items(ALL_TRAINING_FILENAMES))
else:
    print(count_data_items(ALL_TRAINING_FILENAMES))

# Datasets pipeline

In [ ]:
DIM = P['DIM']
MASK_DIM = P['MASK_DIM']
def _parse_only_mask_function(example_proto, to_float=True):
    single_example = tf.io.parse_single_example(example_proto, {'mask': tf.io.FixedLenFeature([], tf.string)})
    mask =  tf.reshape(tf.io.decode_raw(single_example['mask'],out_type='bool'),(MASK_DIM,MASK_DIM,1))
    if to_float:
        return tf.cast(mask, tf.float32)
    
    return mask

def load_only_masks_dataset(filenames, ordered=False, to_float=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(lambda ex: _parse_only_mask_function(ex, to_float= to_float), num_parallel_calls=AUTO)
    return dataset


def _parse_image_function(example_proto,augment = True):
    image_feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'mask': tf.io.FixedLenFeature([], tf.string)
    }
    single_example = tf.io.parse_single_example(example_proto, image_feature_description)
    image = tf.reshape( tf.io.decode_raw(single_example['image'],out_type=np.dtype('uint8')), (DIM,DIM, 3))
    mask =  tf.reshape(tf.io.decode_raw(single_example['mask'],out_type='bool'),(MASK_DIM,MASK_DIM,1))
    
    if augment: # https://www.kaggle.com/kool777/training-hubmap-eda-tf-keras-tpu

        if tf.random.uniform(()) > 0.5:
            image = tf.image.flip_left_right(image)
            mask = tf.image.flip_left_right(mask)

        if tf.random.uniform(()) > 0.4:
            image = tf.image.flip_up_down(image)
            mask = tf.image.flip_up_down(mask)

        if tf.random.uniform(()) > 0.5:
            image = tf.image.rot90(image, k=1)
            mask = tf.image.rot90(mask, k=1)

        if tf.random.uniform(()) > 0.45:
            image = tf.image.random_saturation(image, 0.7, 1.3)

        if tf.random.uniform(()) > 0.45:
            image = tf.image.random_contrast(image, 0.8, 1.2)
    
    return tf.cast(image, tf.float32),tf.cast(mask, tf.float32)

def load_dataset(filenames, ordered=False, augment = True):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(lambda ex: _parse_image_function(ex, augment = augment), num_parallel_calls=AUTO)
    return dataset

def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(128, seed = P['SEED'])
    dataset = dataset.batch(BATCH_SIZE,drop_remainder=True)
    dataset = dataset.prefetch(AUTO)
    return dataset

def get_validation_dataset(ordered=True):
    dataset = load_dataset(VALIDATION_FILENAMES, ordered=ordered, augment = False)
    dataset = dataset.batch(BATCH_SIZE,drop_remainder=True)
    #dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO)
    return dataset

# Model

In [ ]:
# https://tensorlayer.readthedocs.io/en/latest/_modules/tensorlayer/cost.html#dice_coe
def dice_coe(output, target, axis = None, smooth=1e-10):
    output = tf.dtypes.cast( tf.math.greater(output, 0.5), tf. float32 )
    target = tf.dtypes.cast( tf.math.greater(target, 0.5), tf. float32 )
    inse = tf.reduce_sum(output * target, axis=axis)
    l = tf.reduce_sum(output, axis=axis)
    r = tf.reduce_sum(target, axis=axis)

    dice = (2. * inse + smooth) / (l + r + smooth)
    dice = tf.reduce_mean(dice, name='dice_coe')
    return dice

# https://www.kaggle.com/kool777/training-hubmap-eda-tf-keras-tpu
def tversky(y_true, y_pred, alpha=0.7, beta=0.3, smooth=1):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1 - y_pred_pos))
    false_pos = K.sum((1 - y_true_pos) * y_pred_pos)
    return (true_pos + smooth) / (true_pos + alpha * false_neg + beta * false_pos + smooth)
def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true, y_pred)
def focal_tversky_loss(y_true, y_pred, gamma=0.75):
    tv = tversky(y_true, y_pred)
    return K.pow((1 - tv), gamma)

get_custom_objects().update({"focal_tversky": focal_tversky_loss})

In [ ]:
fold = KFold(n_splits=P['NFOLDS'], shuffle=True, random_state=P['SEED'])
for fold,(tr_idx, val_idx) in enumerate(fold.split(ALL_TRAINING_FILENAMES)):
    print(tr_idx, val_idx)
    TRAINING_FILENAMES = [ALL_TRAINING_FILENAMES[fi] for fi in tr_idx]
    VALIDATION_FILENAMES = [ALL_TRAINING_FILENAMES[fi] for fi in val_idx]
    ff = [os.path.basename(fname) for fname in VALIDATION_FILENAMES]
    ff = [fname[:fname.index('-')] for fname in ff]
    print(ff)
    print(count_data_items(TRAINING_FILENAMES)//BATCH_SIZE)
    print(count_data_items(VALIDATION_FILENAMES)//BATCH_SIZE)

In [ ]:
def create_pos_neg_df(file_names):
    img_size = DIM ** 2
    df = pd.DataFrame(0,index=[os.path.basename(fname) for fname in file_names],columns=['pos','neg','total'])
    for fname in file_names:
        print(os.path.basename(fname),fname)
        pos = 0
        total=0
        for m in load_only_masks_dataset(fname):
            pos += tf.math.count_nonzero(m)
            total+=1
    
        pos = pos.numpy()
        total = total * img_size
        df.loc[os.path.basename(fname)] += [pos,total-pos,total]
    
    return df

def calc_pos_neg_weights(filenames):
    names = [os.path.basename(fname) for fname in filenames]
    df_sum = pos_neg_df.loc[names].sum()
    pos_weight = (1 / df_sum['pos'])*(df_sum['total'])/2.0
    neg_weight = (1 / df_sum['neg'])*(df_sum['total'])/2.0 
    return pos_weight.astype(np.float32), neg_weight.astype(np.float32)

# file_names = ALL_TRAINING_FILENAMES
# if P['OVERLAPP']:
#     file_names = file_names + ALL_TRAINING_FILENAMES2
# pos_neg_df = create_pos_neg_df(file_names)

# init_bias = pos_neg_df.sum()
# init_bias = np.array([np.log(init_bias['pos']/init_bias['neg'])],dtype=np.float32)
# init_bias

In [ ]:
class FocalLoss(tf.keras.losses.Loss):
    def __init__(self,alpha=0.25,gamma=2.,epsilon=1e-7,name='focal_loss'):
        super().__init__(name=name)
        self.alpha = tf.constant(alpha, dtype=tf.float32)
        self.alpha_comp = tf.constant(1. - alpha, dtype=tf.float32)
        self.gamma = tf.constant(gamma, dtype=tf.float32)
        self.epsilon = tf.constant(epsilon, dtype=tf.float32)

    def call(self, y_true, y_pred):
        pre = tf.clip_by_value(y_pred,self.epsilon,1. - self.epsilon)
        pos_weight = self.alpha * tf.math.pow(1. - pre,self.gamma)
        neg_weight = self.alpha_comp * tf.math.pow(pre,self.gamma)
        log_weight = (neg_weight + (pos_weight - neg_weight) * y_true)
        return tf.math.add(neg_weight * (1. - y_true) * y_pred._keras_logits,
                   log_weight * (tf.math.log1p(tf.math.exp(-tf.math.abs(y_pred._keras_logits))) + tf.nn.relu(-y_pred._keras_logits)),
                   name=self.name)
    
    
    def get_config(self):
        config = {}
        for k, v in {'alpha': self.alpha, 'gamma': self.gamma,'epsilon': self.epsilon}.items():
            config[k] = K.eval(v) if tf.is_tensor(v) else v
        base_config = super().get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
# def create_model():
#     inp = tf.keras.Input(shape=(512,512,3))
#     unet = sm.Unet(P['BACKBONE'], encoder_weights='imagenet',input_shape=(512,512,3),classes=4)
#     x = unet(inp)
#     x = tf.image.transpose(x)
#     x = tf.keras.layers.Reshape((512,1024,2))(x)
#     x = tf.image.transpose(x)
#     x = tf.keras.layers.Reshape((1024,1024,1))(x)
#     return tf.keras.Model(inp,x)
# #tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
def create_model():
    inp = tf.keras.Input(shape=(512,512,3))
    unet = sm.Unet(P['BACKBONE'], encoder_weights='imagenet',input_shape=(512,512,3))
    x = unet(inp)
    x = tf.image.resize(x,(1024,1024))
    return tf.keras.Model(inp,x)

# Model fit

In [ ]:
# print('#'*35); print('############ FOLD ',fold+1,' #############'); print('#'*35);
# print(f'Image Size: {DIM}, Batch Size: {BATCH_SIZE}')
    
#     # CREATE TRAIN AND VALIDATION SUBSETS
# TRAINING_FILENAMES = ALL_TRAINING_FILENAMES+ ALL_TRAINING_FILENAMES2
# VALIDATION_FILENAMES = TRAINING_FILENAMES
# STEPS_PER_EPOCH = P['STEPS_COE'] * count_data_items(TRAINING_FILENAMES) // BATCH_SIZE
# print('STEPS_PER_EPOCH',STEPS_PER_EPOCH)    

# K.clear_session()
# with strategy.scope():   
#     model = sm.Unet(P['BACKBONE'], encoder_weights='imagenet',decoder_filters=(256, 128, 64, 32, 32,16))

#     model.compile(optimizer = tf.keras.optimizers.Adam(lr = P['LR']),
#                       loss = tf.keras.losses.BinaryCrossentropy(),#'focal_tversky',
#                       metrics=[dice_coe,'accuracy'])
        
#     # CALLBACKS
# checkpoint = tf.keras.callbacks.ModelCheckpoint('/kaggle/working/model-all.h5',
#                                  verbose=P['VERBOSE'],monitor='val_dice_coe',patience = 10,
#                                  mode='max',save_best_only=True)
    
# early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_dice_coe',mode = 'max', patience=10, restore_best_weights=True)
# reduce = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, min_lr=0.00001)
        
# history = model.fit(
#         get_training_dataset(),
#         epochs = P['EPOCHS'],
#         steps_per_epoch = STEPS_PER_EPOCH,
#         callbacks = [checkpoint, reduce,early_stop],
#         validation_data = get_validation_dataset(),
#         verbose=P['VERBOSE']
#     )   
    
#     #with strategy.scope():
#     #    model = tf.keras.models.load_model('/kaggle/working/model-fold-%i.h5'%fold, custom_objects = {"dice_coe": dice_coe})
    
#     # SAVE METRICS
    
#     # PLOT TRAINING
#     # https://www.kaggle.com/cdeotte/triple-stratified-kfold-with-tfrecords
# if P['DISPLAY_PLOT']:        
#     plt.figure(figsize=(15,5))
#     n_e = np.arange(len(history.history['dice_coe']))
#     plt.plot(n_e,history.history['dice_coe'],'-o',label='Train dice_coe',color='#ff7f0e')
#     plt.plot(n_e,history.history['val_dice_coe'],'-o',label='Val dice_coe',color='#1f77b4')
#     x = np.argmax( history.history['val_dice_coe'] ); y = np.max( history.history['val_dice_coe'] )
#     xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
#     plt.scatter(x,y,s=200,color='#1f77b4'); plt.text(x-0.03*xdist,y-0.13*ydist,'max dice_coe\n%.2f'%y,size=14)
#     plt.ylabel('dice_coe',size=14); plt.xlabel('Epoch',size=14)
#     plt.legend(loc=2)
#     plt2 = plt.gca().twinx()
#     plt2.plot(n_e,history.history['loss'],'-o',label='Train Loss',color='#2ca02c')
#     plt2.plot(n_e,history.history['val_loss'],'-o',label='Val Loss',color='#d62728')
#     x = np.argmin( history.history['val_loss'] ); y = np.min( history.history['val_loss'] )
#     ydist = plt.ylim()[1] - plt.ylim()[0]
#     plt.scatter(x,y,s=200,color='#d62728'); plt.text(x-0.03*xdist,y+0.05*ydist,'min loss',size=14)
#     plt.ylabel('Loss',size=14)
#     plt.legend(loc=3)
#     plt.show()

In [ ]:
from datetime import datetime
M = {}
metrics = ['loss','dice_coe','accuracy']
for fm in metrics:
    M['val_'+fm] = []

fold = KFold(n_splits=P['NFOLDS'], shuffle=True, random_state=P['SEED'])
for fold,(tr_idx, val_idx) in enumerate(fold.split(ALL_TRAINING_FILENAMES)):
    start_time = datetime.now()
    print('#'*35); print('############ FOLD ',fold+1,' #############'); print('#'*35);
    print(f'Image Size: {DIM}, Batch Size: {BATCH_SIZE}')
    print('Start time:',start_time)
    # CREATE TRAIN AND VALIDATION SUBSETS
    TRAINING_FILENAMES = [ALL_TRAINING_FILENAMES[fi] for fi in tr_idx]
    if P['OVERLAPP']:
        TRAINING_FILENAMES += [ALL_TRAINING_FILENAMES2[fi] for fi in tr_idx]
    
    VALIDATION_FILENAMES = [ALL_TRAINING_FILENAMES[fi] for fi in val_idx]
    STEPS_PER_EPOCH = P['STEPS_COE'] * count_data_items(TRAINING_FILENAMES) // BATCH_SIZE
    
    # BUILD MODEL
    K.clear_session()
    with strategy.scope():   
#         model = sm.Unet(P['BACKBONE'], encoder_weights='imagenet', decoder_filters=(256, 128, 64, 32, 32,16))
        model = create_model()
        model.compile(optimizer = tf.keras.optimizers.Adam(lr = P['LR']),
                      loss = tf.keras.losses.BinaryCrossentropy(),#'focal_tversky',
                      metrics=[dice_coe,'accuracy'])
        
    # CALLBACKS
    checkpoint = tf.keras.callbacks.ModelCheckpoint('/kaggle/working/model-fold-%i.h5'%fold,
                                 verbose=P['VERBOSE'],monitor='val_dice_coe',patience = 10,
                                 mode='max',save_best_only=True)
    
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_dice_coe',mode = 'max', patience=10, restore_best_weights=True)
    reduce = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=0.00001, verbose=P['VERBOSE'])
#     reduce = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
    
        
    print(f'Training Model Fold {fold+1}...')
    history = model.fit(
        get_training_dataset(),
        epochs = P['EPOCHS'],
        steps_per_epoch = STEPS_PER_EPOCH,
        callbacks = [checkpoint, reduce,early_stop],
        validation_data = get_validation_dataset(),
        verbose=P['VERBOSE']
    )
    
    print('Total time:',datetime.now() - start_time)
    tf.keras.models.save_model(model,'/kaggle/working/last-model-fold-%i-%i.h5'% (fold,len(history.history['dice_coe'])))
    #with strategy.scope():
    #    model = tf.keras.models.load_model('/kaggle/working/model-fold-%i.h5'%fold, custom_objects = {"dice_coe": dice_coe})
    
    # SAVE METRICS
    m = model.evaluate(get_validation_dataset(),return_dict=True)
    for fm in metrics:
        M['val_'+fm].append(m[fm])
    
    # PLOT TRAINING
    # https://www.kaggle.com/cdeotte/triple-stratified-kfold-with-tfrecords
    if P['DISPLAY_PLOT']:        
        plt.figure(figsize=(15,5))
        n_e = np.arange(len(history.history['dice_coe']))
        plt.plot(n_e,history.history['dice_coe'],'-o',label='Train dice_coe',color='#ff7f0e')
        plt.plot(n_e,history.history['val_dice_coe'],'-o',label='Val dice_coe',color='#1f77b4')
        x = np.argmax( history.history['val_dice_coe'] ); y = np.max( history.history['val_dice_coe'] )
        xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
        plt.scatter(x,y,s=200,color='#1f77b4'); plt.text(x-0.03*xdist,y-0.13*ydist,'max dice_coe\n%.2f'%y,size=14)
        plt.ylabel('dice_coe',size=14); plt.xlabel('Epoch',size=14)
        plt.legend(loc=2)
        plt2 = plt.gca().twinx()
        plt2.plot(n_e,history.history['loss'],'-o',label='Train Loss',color='#2ca02c')
        plt2.plot(n_e,history.history['val_loss'],'-o',label='Val Loss',color='#d62728')
        x = np.argmin( history.history['val_loss'] ); y = np.min( history.history['val_loss'] )
        ydist = plt.ylim()[1] - plt.ylim()[0]
        plt.scatter(x,y,s=200,color='#d62728'); plt.text(x-0.03*xdist,y+0.05*ydist,'min loss',size=14)
        plt.ylabel('Loss',size=14)
        plt.legend(loc=3)
        plt.show()

In [ ]:
# history = model.fit(
#         get_training_dataset(),
#         epochs = 5,
#         steps_per_epoch = STEPS_PER_EPOCH,
#         initial_epoch = 2,
#         callbacks = [checkpoint, reduce,early_stop],
#         validation_data = get_validation_dataset(),
#         verbose=P['VERBOSE']
#     )

In [ ]:
### WRITE METRICS
import json
from datetime import datetime
M['datetime'] = str(datetime.now())
for fm in metrics:
    M['oof_'+fm] = np.mean(M['val_'+fm])
    print('OOF '+ fm + ' '+ str(M['oof_'+fm]))
with open('metrics.json', 'w') as outfile:
    json.dump(M, outfile)